In [1]:
using Gurobi, LinearAlgebra, Ipopt, CSV, JuMP, DataFrames, Clp, Cbc, Dates

### Model 1: Maximum coverage with a limit of areas that can be covered, x and y not within .01 of existing electricity

In [21]:
function model1opt(max_units = 25, min_dist = 1, filter_density = .025)
    model1 = Model(Cbc.Optimizer)

    df= CSV.read("data.csv", copycols=true)
    electricity = CSV.read("electricity.csv", copycols=true)
    fdf = filter(row -> row[:value] > filter_density, df)

    m = size(fdf)[1]
    n = size(electricity)[1]

    @variable(model1, b[1:m], Bin)

    @constraint(model1,sum(b[i] for i=1:m)<=max_units)

    for i=1:m
        for j=1:n
            @constraint(model1, sqrt((fdf[7][i] - electricity[2][j])^2+(fdf[8][i] - electricity[3][j])^2) >= min_dist*b[i])
        end
    end

    @objective(model1, Max, sum(fdf[9][i]*b[i] for i = 1:m))

    model1;
    
    optimize!(model1)
    obj_val = objective_value(model1)
    
    soln = value.(b)
    sol = [i for (i,n) in enumerate(soln) if n == 1]
    
    return (max_units, min_dist, filter_density, obj_val, length(sol), sol)
end

model1opt (generic function with 4 methods)

In [ ]:
filter_list = [.005 .01 .025]
max_val_list = [10 20 25 50]
min_dist = [.1 .25 .5 .75 1]

results1 = []

for mval in max_val_list
    println("mval:", mval, Dates.Time(Dates.now()))
    for mdis in min_dist
        println("mdis", mdis, Dates.Time(Dates.now()))
        for f in filter_list
            println("f", f, Dates.Time(Dates.now()))
            push!(results1, model1opt(mval, mdis, f))
        end
    end
end

rdf1 = DataFrame(results1)

CSV.write("method2.csv", rdf1)

mval:1022:32:29.641
mdis0.122:32:29.641f0.00522:32:29.641


┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `geti

f0.0122:33:40.991
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 2.56402 - 0.08 seconds
Cgl0004I processed model has 1 rows, 627 columns (627 integer (627 of which binary)) and 627 elements
Cbc0012I Integer solution of -2.5538178 found by DiveCoefficient after 0 iterations and 0 nodes (0.35 seconds)
Cbc0001I Search completed - best objective -2.553817829, took 0 iterations and 0 nodes (0.35 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -2.55382 to -2.55382
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried

mdis0.522:35:45.549f0.00522:35:45.549
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 2.5579 - 0.01 seconds
Cgl0004I processed model has 1 rows, 46 columns (46 integer (46 of which binary)) and 46 elements
Cbc0012I Integer solution of -2.5538178 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 seconds)
Cbc0001I Search completed - best objective -2.553817829, took 0 iterations and 0 nodes (0.02 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -2.55382 to -2.55382
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
C

f0.02522:38:50.611
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 2.54968 - 0.02 seconds
Cgl0004I processed model has 1 rows, 129 columns (129 integer (129 of which binary)) and 129 elements
Cbc0012I Integer solution of -2.1716635 found by DiveCoefficient after 0 iterations and 0 nodes (0.08 seconds)
Cbc0001I Search completed - best objective -2.171663518, took 0 iterations and 0 nodes (0.08 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -2.17166 to -2.17166
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was trie

f0.0122:41:33.518
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 3.43324 - 0.08 seconds
Cgl0004I processed model has 1 rows, 627 columns (627 integer (627 of which binary)) and 627 elements
Cbc0012I Integer solution of -3.412241 found by DiveCoefficient after 0 iterations and 0 nodes (0.32 seconds)
Cbc0001I Search completed - best objective -3.412241007, took 0 iterations and 0 nodes (0.32 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -3.41224 to -3.41224
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 

f0.0122:44:34.257
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 3.40239 - 0.08 seconds
Cgl0004I processed model has 1 rows, 443 columns (443 integer (443 of which binary)) and 443 elements
Cbc0012I Integer solution of -3.3690858 found by DiveCoefficient after 0 iterations and 0 nodes (0.28 seconds)
Cbc0001I Search completed - best objective -3.369085788, took 0 iterations and 0 nodes (0.28 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -3.36909 to -3.36909
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried

f0.0122:47:35.52
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 3.04597 - 0.08 seconds
Cgl0004I processed model has 1 rows, 184 columns (184 integer (184 of which binary)) and 184 elements
Cbc0012I Integer solution of -1.0136182 found by DiveCoefficient after 0 iterations and 0 nodes (0.26 seconds)
Cbc0001I Search completed - best objective -1.013618212, took 0 iterations and 0 nodes (0.26 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -1.01362 to -1.01362
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 

### Iteration 1: max_val = 25, filter .025, distance .01, .05,
3.7106751760000005; [1, 4, 11, 12, 13, 14, 15, 16, 19, 20, 22, 23, 24, 25, 27, 28, 29, 30, 31, 39, 40, 43, 44, 45, 48]
### Iteration 2: max_val = 25, filter .025, distance .1, .2, 
3.6787334210000004; [4, 11, 12, 13, 14, 15, 16, 17, 19, 20, 22, 23, 24, 27, 28, 29, 30, 31, 36, 39, 40, 43, 44, 45, 48]
### Iteration 3: max_val = 25, filter .025, distance .3
3.6484393790000005; [4, 11, 12, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 27, 28, 29, 30, 31, 32, 39, 40, 43, 44, 45, 48]
### Iteration 3: max_val = 25, filter .025, distance .4, .5
3.593442029; [4, 11, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 27, 28, 29, 30, 31, 32, 39, 40, 43, 44, 45, 48, 49]
### Iteration 4: max_val = 25, filter .025, distance .75
2.8700284710000004; [4, 8, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 27, 28, 32, 33, 34, 40, 41, 42, 43, 44, 45, 46, 49]
### Iteration 5: max_val = 25, filter .025, distance .8
2.808501664; [4, 5, 8, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 27, 28, 32, 34, 35, 41, 43, 44, 45, 46, 49, 50]
### Iteration 6: max_val = 25, filter .025, distance .9
1.7229052080000002; [7, 14, 15, 16, 19, 22, 26, 27, 32, 34, 35, 38, 41, 43, 44, 45, 46, 50]
### Iteration 7: max_val = 25, filter .025, distance 1
0.8715401300000002; [15, 16, 26, 32, 34, 35, 38, 41, 43, 44, 45, 46, 50]

### Model 2: Maximum summation of distance and coverage.

In [6]:
function model2opt(max_units = 25, min_dist = 1, filter_density = .025)
    model2 = Model(Cbc.Optimizer)

    df= CSV.read("data.csv", copycols=true)

    fdf = filter(row -> row[:value] > filter_density, df)
    electricity = CSV.read("electricity.csv", copycols=true)
    
    m = size(fdf)[1]
    n = size(electricity)[1]

    @variable(model2, b[1:m], Bin)

    @constraint(model2,sum(b[i] for i=1:m)<=max_units)

    for i=1:m
        for j=1:n
            @constraint(model2, sqrt((fdf[7][i] - electricity[2][j])^2+(fdf[8][i] - electricity[3][j])^2) >= min_dist*b[i])
        end
    end

    @objective(model2, Max, sum((fdf[9][i]+ minimum([sqrt((fdf[7][i] - electricity[2][j])^2+(fdf[8][i] - electricity[3][j])^2) for j=1:n]))*b[i] for i=1:m))
    model2;
    
    optimize!(model2)
    obj_val = objective_value(model2)
    
    soln = value.(b)
    sol = [i for (i,n) in enumerate(soln) if n == 1]
    
    return (max_units, min_dist, filter_density, obj_val, length(sol), sol)
end

model2opt (generic function with 4 methods)

In [7]:
filter_list = [.005 .01 .025]
max_val_list = [10 20 25 50]
min_dist = [.1 .25 .5 .75 1]

results = []

for mval in max_val_list
    println("mval:", mval, Dates.Time(Dates.now()))
    for mdis in min_dist
        print("mdis", mdis, Dates.Time(Dates.now()))
        for f in filter_list
            println("f", f, Dates.Time(Dates.now()))
            push!(results, model2opt(mval, mdis, f))
        end
    end
end

mval:1018:35:30.274
mdis0.118:35:30.426f0.00518:35:30.486


┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `geti

f0.0118:37:53.908
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 17.6707 - 0.08 seconds
Cgl0004I processed model has 1 rows, 627 columns (627 integer (627 of which binary)) and 627 elements
Cbc0012I Integer solution of -17.67075 found by DiveCoefficient after 0 iterations and 0 nodes (0.33 seconds)
Cbc0001I Search completed - best objective -17.67074973890588, took 0 iterations and 0 nodes (0.33 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -17.6707 to -17.6707
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was 

mdis0.518:42:15.218f0.00518:42:15.219
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 14.6983 - 0.01 seconds
Cgl0004I processed model has 1 rows, 46 columns (46 integer (46 of which binary)) and 46 elements
Cbc0012I Integer solution of -14.698255 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 seconds)
Cbc0001I Search completed - best objective -14.69825514167977, took 0 iterations and 0 nodes (0.02 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -14.6983 to -14.6983
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 sec

f0.02518:48:49.882
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 17.2079 - 0.03 seconds
Cgl0004I processed model has 1 rows, 129 columns (129 integer (129 of which binary)) and 129 elements
Cbc0012I Integer solution of -17.207881 found by DiveCoefficient after 0 iterations and 0 nodes (0.09 seconds)
Cbc0001I Search completed - best objective -17.20788145642142, took 0 iterations and 0 nodes (0.09 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -17.2079 to -17.2079
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique wa

...

f0.0119:14:49.282
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 42.5875 - 0.08 seconds
Cgl0004I processed model has 1 rows, 561 columns (561 integer (561 of which binary)) and 561 elements
Cbc0012I Integer solution of -42.587523 found by DiveCoefficient after 0 iterations and 0 nodes (0.32 seconds)
Cbc0001I Search completed - best objective -42.58752296959377, took 0 iterations and 0 nodes (0.32 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -42.5875 to -42.5875
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was

f0.0119:21:21.81
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 42.5875 - 0.09 seconds
Cgl0004I processed model has 1 rows, 310 columns (310 integer (310 of which binary)) and 310 elements
Cbc0012I Integer solution of -42.587523 found by DiveCoefficient after 0 iterations and 0 nodes (0.29 seconds)
Cbc0001I Search completed - best objective -42.58752296959377, took 0 iterations and 0 nodes (0.29 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -42.5875 to -42.5875
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was 

f0.0119:27:51.738
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 81.0961 - 0.07 seconds
Cgl0004I processed model has 1 rows, 627 columns (627 integer (627 of which binary)) and 627 elements
Cbc0012I Integer solution of -81.096144 found by DiveCoefficient after 0 iterations and 0 nodes (0.32 seconds)
Cbc0001I Search completed - best objective -81.09614424355753, took 0 iterations and 0 nodes (0.32 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -81.0961 to -81.0961
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was

f0.0119:34:34.531
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 81.0961 - 0.08 seconds
Cgl0004I processed model has 1 rows, 443 columns (443 integer (443 of which binary)) and 443 elements
Cbc0012I Integer solution of -81.096144 found by DiveCoefficient after 0 iterations and 0 nodes (0.29 seconds)
Cbc0001I Search completed - best objective -81.09614424355753, took 0 iterations and 0 nodes (0.29 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -81.0961 to -81.0961
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was

f0.0119:54:30.46
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 81.0961 - 0.09 seconds
Cgl0004I processed model has 1 rows, 184 columns (184 integer (184 of which binary)) and 184 elements
Cbc0012I Integer solution of -81.096144 found by DiveCoefficient after 0 iterations and 0 nodes (0.29 seconds)
Cbc0001I Search completed - best objective -81.09614424355753, took 0 iterations and 0 nodes (0.29 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from -81.0961 to -81.0961
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was 

In [9]:
results

60-element Array{Any,1}:
 (10, 0.1, 0.005, 17.670749738905883, 10, [373, 374, 400, 413, 414, 433, 434, 456, 457, 674])                                                     
 (10, 0.1, 0.01, 17.20788145642142, 10, [110, 111, 120, 128, 133, 134, 146, 167, 168, 212])                                                       
 (10, 0.1, 0.025, 14.698255141679772, 10, [16, 22, 26, 32, 38, 41, 43, 44, 45, 46])                                                               
 (10, 0.25, 0.005, 17.670749738905883, 10, [373, 374, 400, 413, 414, 433, 434, 456, 457, 674])                                                    
 (10, 0.25, 0.01, 17.20788145642142, 10, [110, 111, 120, 128, 133, 134, 146, 167, 168, 212])                                                      
 (10, 0.25, 0.025, 14.698255141679772, 10, [16, 22, 26, 32, 38, 41, 43, 44, 45, 46])                                                              
 (10, 0.5, 0.005, 17.670749738905883, 10, [373, 374, 400, 413, 414, 433, 434, 456, 457, 674])

In [11]:
rdf = DataFrame(results)

,1,2,3,4,5,6
,Int64,Float64,Float64,Float64,Int64,Array…
1,10,0.1,0.005,17.6707,10,"[373, 374, 400, 413, 414, 433, 434, 456, 457, 674]"
2,10,0.1,0.01,17.2079,10,"[110, 111, 120, 128, 133, 134, 146, 167, 168, 212]"
3,10,0.1,0.025,14.6983,10,"[16, 22, 26, 32, 38, 41, 43, 44, 45, 46]"
4,10,0.25,0.005,17.6707,10,"[373, 374, 400, 413, 414, 433, 434, 456, 457, 674]"
5,10,0.25,0.01,17.2079,10,"[110, 111, 120, 128, 133, 134, 146, 167, 168, 212]"
6,10,0.25,0.025,14.6983,10,"[16, 22, 26, 32, 38, 41, 43, 44, 45, 46]"
7,10,0.5,0.005,17.6707,10,"[373, 374, 400, 413, 414, 433, 434, 456, 457, 674]"
8,10,0.5,0.01,17.2079,10,"[110, 111, 120, 128, 133, 134, 146, 167, 168, 212]"
9,10,0.5,0.025,14.6983,10,"[16, 22, 26, 32, 38, 41, 43, 44, 45, 46]"


In [20]:
CSV.write("method2.csv", rdf)

"method2.csv"